# Extracción de información de imágenes

En este cuaderno veremos algunas herramientas para extracción de información de imágenes.

Haremos uso de las librerías numpy, openCV y scikit image, además de matplotlib para mostrar gráficos de histogramas de color

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import skimage.feature as feature


Definimos una función para mostrar el histograma de color independiente del espacio.

In [ ]:
def plot_histogram(image):
  pixels = [image[:,:,i].flatten() for i in range(3)]
  plt.hist(pixels[0], bins=256, density=False, color='red', alpha=0.5)
  plt.hist(pixels[1], bins=256, density=False, color='green', alpha=0.4)
  plt.hist(pixels[2], bins=256, density=False, color='blue', alpha=0.3)

  plt.show()

Cargamos la imagen inicial, la cual mostramos en pantalla. También calculamos histograma de color en el espacio RGB.

In [ ]:
imagefile = "madeleine.jpg"

In [ ]:
image = cv2.imread(imagefile)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(image)
plot_histogram(image)

Mostramos una representación y el histograma de color en el espacio HSB



In [ ]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
cv2_imshow(hsv_image)
plot_histogram(hsv_image)

De igual manera, mostramos una representación e histograma en el espacio lab vemos que en este caso el color mucho más plano y será más difícil extraer una representación.


In [ ]:
lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
cv2_imshow(lab_image)
plot_histogram(lab_image)


Al igual que la anterior, la representación parece tener poca variabilidad lo cual podría afectar la buena representación de esta imagen.

In [ ]:
ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
cv2_imshow(ycrcb_image)
plot_histogram(ycrcb_image)

Hacemos uso de la librería scikit image para calcular las propiedades de GLCM de toda la imagen.

In [ ]:
graycom = feature.graycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)

contrast = feature.graycoprops(graycom, 'contrast')
dissimilarity = feature.graycoprops(graycom, 'dissimilarity')
homogeneity = feature.graycoprops(graycom, 'homogeneity')
energy = feature.graycoprops(graycom, 'energy')
correlation = feature.graycoprops(graycom, 'correlation')
ASM = feature.graycoprops(graycom, 'ASM')

print("Contrast: {}".format(contrast))
print("Dissimilarity: {}".format(dissimilarity))
print("Homogeneity: {}".format(homogeneity))
print("Energy: {}".format(energy))
print("Correlation: {}".format(correlation))
print("ASM: {}".format(ASM))

Utilizamos el detector de esquinas de Harris para encontrar elementos de interés. Encontramos que identifica de manera correcta ojos y boca. Sin embargo por las texturas de la vestimenta, encuentra una serie de esquinas en esos elementos.

In [ ]:
img = image.copy()
dst = cv2.cornerHarris(gray,2,3,0.04)
#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.10*dst.max()]=[255,0,0]
cv2_imshow(img)